# Testing screencapture with different libraries
   (openCV, PyAutoGUI, Pillow, PyWin32)
    
### References:
Screen capture:
* https://stackoverflow.com/questions/24129253/screen-capture-with-opencv-and-python-2-7
* https://www.youtube.com/watch?v=WymCpVUPWQ4&ab_channel=LearnCodeByGaming
* https://www.youtube.com/watch?v=ks4MPfMq8aQ&ab_channel=sentdex
* https://stackoverflow.com/questions/3586046/fastest-way-to-take-a-screenshot-with-python-on-windows
* https://stackoverflow.com/questions/41785831/how-to-optimize-conversion-from-pycbitmap-to-opencv-image

Window listing:
* https://stackoverflow.com/questions/55547940/how-to-get-a-list-of-the-name-of-every-open-window

### Importing Libraries
Note: Original PIL (Python Imaging Library) was discontinued since 2011, with Pillow taking it's place.

In [32]:
import cv2 as cv
import numpy as np
from time import time
img_counter = 0
#import pyautogui # needs to be installed first
#from mms import mms # needs to be installed first



### Main Loop (With Pillow):

In [7]:
from PIL import ImageGrab   # This is actually Pillow, but it's still imported as PIL. 
counter = 1
total = 0

while(True):
    prev_time = time()
    
    #take the screenshot, bbox defines a boundry box for it
    #bbox = (0, 0, 320, 240)
    bbox = None
    img_pil = ImageGrab.grab(bbox)
    #transform to an array and reshape it into a matrix for openCV
    img_np = np.array(img_pil, dtype='uint8').reshape((img_pil.size[1], img_pil.size[0],3))
    
    img_np = cv.cvtColor(img_np, cv.COLOR_RGB2BGR)
    
    cv.imshow('window',img_np)
    #print('FPS {}'.format(1 / (time() - prev_time)))
    total += 1 / (time() - prev_time)
    print('Avg FPS {}'.format(total/counter))
    counter += 1
    if cv.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break

Avg FPS 6.604859904918327
Avg FPS 10.68076658709789
Avg FPS 10.169758579549848
Avg FPS 10.721239239163515
Avg FPS 11.35425313291984
Avg FPS 11.42257305091403
Avg FPS 11.745121064082841
Avg FPS 12.146313617867415
Avg FPS 12.33592638184569
Avg FPS 12.669021078842169
Avg FPS 12.74570405230562
Avg FPS 12.567518784835954
Avg FPS 12.655392543921662
Avg FPS 12.863189556783851
Avg FPS 13.050101809785627
Avg FPS 13.204813507720235
Avg FPS 13.30767047283291
Avg FPS 13.400300124071364
Avg FPS 13.372802932184001
Avg FPS 13.419131028557747
Avg FPS 13.368081109104859
Avg FPS 13.46838563304528
Avg FPS 13.585668293515857
Avg FPS 13.591906238482062
Avg FPS 13.66121724487637
Avg FPS 13.787420700814218
Avg FPS 13.849459669593072
Avg FPS 13.931225059930009
Avg FPS 13.991630503242279
Avg FPS 14.002190756189679
Avg FPS 13.992745400312648
Avg FPS 13.969740392297867
Avg FPS 13.97255074340797
Avg FPS 13.980121088016618
Avg FPS 14.019511916751648
Avg FPS 13.95122140781015
Avg FPS 13.968091047233782
Avg FPS 14.0

#### Pillow results:
The framerate seems awfully slow, and reducing the boundry box to ridiculously small sizes barely seems to help.


### With Windows API:

In [53]:
import win32gui
import win32ui
import win32con

def list_all_open_windows():
    def winEnumHandler( hwnd, ctx ):
        if win32gui.IsWindowVisible( hwnd ):
            print (hex(hwnd), win32gui.GetWindowText( hwnd ))

    win32gui.EnumWindows(winEnumHandler, None)

#list_all_open_windows()

def window_capture():
    #w = 1920 # set this
    #h = 1080 # set this
    bmpfilenamename = "sample" + str(img_counter) + ".jpg" #set this

    hwnd = win32gui.FindWindow(None, 'Samurai Gunn')
    #hwnd = None
    
    # Getting the window's size and accounting for window screenshot borders
    window_rect = win32gui.GetWindowRect(hwnd)
    titlebar_px = 38
    border_px = 9
    #titlebar_px = 0
    #border_px = 0
    #print(window_rect)
    #w = window_rect[2] - window_rect[0] - (2 * border_px)
    #h = window_rect[3] - window_rect[1] - titlebar_px - border_px
    #print(h,w)
    
    # For samurai Gunn, the non-fullscreen dimensions should be:
    w = 320
    h = 240
    
    crop_x = border_px
    crop_y = titlebar_px
    
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (w, h) , dcObj, (crop_x, crop_y), win32con.SRCCOPY)
    
    # saving screenshot to file
    dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)
    
    # Converting to format useful for opencv
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    #img = np.fromstring(signedIntsArray, dtype='uint8')
    # np.fromstring is deprecated, should use np.frombuffer instead
    img = np.frombuffer(signedIntsArray, dtype='uint8')
    img.shape = (h, w, 4)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())
    
    # Dropping alpha channel may be useful for some applications, like cv.matchTemplate()
    # which may throw an error otherwise
    #img = img[...,:3]   # this drops alpha channel 
    
    return img

window_capture()
img_counter += 1

In [13]:
import win32gui
# If we need help to figure out the window name, this funcion will list the names of every open window
def list_all_open_windows():
    def winEnumHandler( hwnd, ctx ):
        if win32gui.IsWindowVisible( hwnd ):
            print (hex(hwnd), win32gui.GetWindowText( hwnd ))

    win32gui.EnumWindows( winEnumHandler, None )

list_all_open_windows()

0x30134 
0x2037e reading_from_screen - Jupyter Notebook - Google Chrome
0x40368 Samurai Gunn
0x2037c Calculator
0x404ce 
0x905fc experiments
0x60686 Settings
0x9069a Settings
0x304da Anaconda Prompt (anaconda3)
0x803f6 Jupyter Notebook (anaconda3)
0x40102 
0x10304 Microsoft Text Input Application
0x101ec 
0x101e4 
0x101c6 
0x101c4 
0x2005e NVIDIA GeForce Overlay
0xa0596 Steam
0x206fc bienvenida-y-reglas - Discord
0x20194 Program Manager


In [12]:
from pywinauto import Desktop

windows = Desktop(backend="uia").windows()
print([w.window_text() for w in windows])

['Taskbar', 'reading_from_screen - Jupyter Notebook - Google Chrome', 'Samurai Gunn', 'Calculator', 'experiments', 'Anaconda Prompt (anaconda3)', 'Jupyter Notebook (anaconda3)', 'NVIDIA GeForce Overlay', 'Steam', 'bienvenida-y-reglas - Discord', 'Program Manager']


## Main Loop (with pywin32)

In [18]:
counter = 1
total = 0

while(True):
    prev_time = time()
    screenshot = window_capture()
    img_procesada = procesar(screenshot)
    
    cv.imshow('Screenshot', screenshot)
    #print('FPS {}'.format(1 / (time() - prev_time)))
     
    #total += 1 / (time() - prev_time)
    #print('Avg FPS {}'.format(total/counter))
    #counter += 1
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break

#### pywin32 results:
When capturing the fullscreen window, the framerate sees an increase of almost 40% (actually closer to 36.8%, still pretty dam good) compared to using pillow! great news!

AAAAND, when capturing the small Samurai Gunn window, it goes to an avg of 44.24 fps! that is almost 200% increase!!!
and uhh... after cropping the black borders it went up to an avg of 790 FPS.... yeah, we are good with this.